In [44]:
import yfinance as yf
import pandas as pd 

In [45]:
datafinance = yf.download("EURUSD=X", start="2023-08-20", end="2023-09-17", interval="15m")

[*********************100%***********************]  1 of 1 completed


In [46]:
datafinance.Open.iloc

In [47]:
datafinance.head(3)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-08-21 00:00:00,1.087666,1.087666,1.087548,1.087548,1.087548,0
2023-08-21 00:15:00,1.087548,1.087666,1.087429,1.087666,1.087666,0
2023-08-21 00:30:00,1.087548,1.087666,1.087429,1.087548,1.087548,0


In [48]:
datafinance.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-09-15 21:45:00,1.066439,1.066553,1.066325,1.066325,1.066325,0
2023-09-15 22:00:00,1.066212,1.066212,1.066212,1.066212,1.066212,0
2023-09-15 22:15:00,1.066212,1.066212,1.066212,1.066212,1.066212,0


In [49]:
##Defining signal function
def signal_generator(df): 
    open = df.Open.iloc[-1]
    close= df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]


    ##Bearish Pattern (selling signal)
    if (open>close and
        previous_open<previous_close and 
        close<previous_open and  
        open>=previous_close):
        return 1 
    

    ##Bullish Pattern (buying signal)
    elif (open<close and 
          previous_open>previous_close and 
          close>previous_open and
          open<+previous_close):
        return 2
    ##No clear pattern 
    else: 
        return 0

In [50]:
signal = []
signal.append(0)
for i in range(1,len(datafinance)):
    df =datafinance[i-1:i+1]
    signal.append(signal_generator(df))
datafinance["signal"] = signal

In [51]:
datafinance.signal.value_counts()

0    1814
1      68
2      14
Name: signal, dtype: int64

In [52]:
from apscheduler.schedulers.blocking import BlockingScheduler

In [53]:
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders 
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [40]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import json
access_token= "f3e0d684xxxxxxxxxxxxxxx"
account_ID= "xxxxxxxxxxxxxx"
client = API(access_token=access_token,environment="live" or "practice")



In [54]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import json
access_token= "f3e0d68xxxxxxxxxxxxxxxx"
account_ID= "xxxxxxxxxxxxx"
def get_candles(n):
    access_token ="f3xxxxxxxxxxxxxxxxxxxxxxxxxxx"
    client= CandleClient(access_token,real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles


candles= get_candles(3)
for candle in candles: 
    print(float(str(candle.bid.o))>1)

True
True
True


In [55]:
def trading_job():
    candles= get_candles(3)
    dfstream= pd.DataFrame(columns=["Open", "Close", "High", "Low"])

    i=0
    for candle in candles: 
        dfstream.loc[i, ["Open"]]=float(str(candle.bid.o))
        dfstream.loc[i, ["Close"]]=float(str(candle.bid.c))
        dfstream.loc[i, ["High"]]=float(str(candle.bid.h))
        dfstream.loc[i, ["Low"]]=float(str(candle.bid.l))
        i=i+1


    dfstream["Open"] = dfstream["Open"].astype(float)
    dfstream["Close"] = dfstream["Close"].astype(float)
    dfstream["High"] = dfstream["High"].astype(float)
    dfstream["Low"] = dfstream["Low"].astype(float)

    signal= signal_generator(dfstream.iloc[:-1,:])


    ####EXECUTING ORDERS

    client=API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream["High"].iloc[-2]-dfstream["Low"].iloc[-2])

    SLBuy = float(str(candle.bid.o)) - previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)

    signal = 2

    ###SELL

    if signal == 1: 
        mo = MarketOrderRequest(instrument="EUR_USD", units =-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data,stopLossOnFill=StopLossDetails(price=SLSell).data)
        r= orders.OrderCreate(accountID="xxxxxxxxxxxx", data=mo.data)
        rv=client.request(r)
        print(rv)


    ###Buy
    elif signal ==2:
        mo = MarketOrderRequest(instrument="EUR_USD", units =1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data,stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID="xxxxxxxxxxxxx", data=mo.data)
        rv = client.request(r)
        print(rv)

        
        

In [ ]:
##EXECUTION
trading_job()
